# load data from Synapse ADI, perform a quaternion rotation.

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [2]:
import synapseclient
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import json

In [3]:
# Login into the Synapse Client

# syn = synapseclient.Synapse()
syn = synapseclient.login()

# Enter login and password
#syn.login('login','password')

# Query demographics table

In [14]:
# demographicstable = syn.tableQuery('select * from syn5511429')

INPUT_DEMO_SYNID = "syn10146552"

demo_syntable = syn.tableQuery("SELECT * FROM syn10146552")
demo = demo_syntable.asDataFrame()
healthCodeList = ", ".join( repr(i) for i in demo["healthCode"]) 


# Query multiple datasets, starting from DeviceMotion

In [15]:
INPUT_WALKING_ACTIVITY_TABLE_SYNID = "syn10146553"
walkingtable = syn.tableQuery(('select * FROM {0} WHERE healthCode IN ({1}) AND "deviceMotion_walking_outbound.json.items" is not null LIMIT 500').format(INPUT_WALKING_ACTIVITY_TABLE_SYNID, healthCodeList))                              
walking_df = walkingtable.asDataFrame()
walking_df['idx'] = walking_df.index
walking_df = walking_df.drop(['createdOn', 'appVersion', 'phoneInfo','accel_walking_outbound.json.items',
                              'pedometer_walking_outbound.json.items','accel_walking_return.json.items',
                              'deviceMotion_walking_return.json.items','pedometer_walking_return.json.items',
                              'accel_walking_rest.json.items'],axis=1)

walking_df = walking_df.dropna()

filePaths_DMoutbound = syn.downloadTableColumns(walkingtable, ['deviceMotion_walking_outbound.json.items'])
filePaths_DMrest = syn.downloadTableColumns(walkingtable, ['deviceMotion_walking_rest.json.items'])

# Store DeviceMotion datsets in dataframes and Pickle them

In [16]:
demographics_df=demographicstable.asDataFrame()

wlist = np.arange(len(walking_df))

DMoutboundPathlist=[filePaths_DMoutbound[str(int(walking_df.ix[entry,'deviceMotion_walking_outbound.json.items']))] for entry in wlist] 
DMrestPathlist = [filePaths_DMrest[str(int(walking_df.ix[entry,'deviceMotion_walking_rest.json.items']))] for entry in wlist]

walking_df['DMoutboundPaths'] = DMoutboundPathlist
walking_df['DMrestPaths'] = DMrestPathlist

demographics_df.to_pickle('demographics_df.pkl')
walking_df.to_pickle('walking_df.pkl')